In [1]:
%classpath add mvn com.salesforce.transmogrifai transmogrifai-core_2.11 0.5.1

In [2]:
%classpath add mvn org.apache.spark spark-mllib_2.11 2.3.0

In [3]:
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkContext
import org.apache.spark.sql.functions.udf

import com.salesforce.op._
import com.salesforce.op.features._
import com.salesforce.op.features.types._
import com.salesforce.op.stages.impl.classification._
import com.salesforce.op.evaluators.Evaluators

import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkContext
import org.apache.spark.sql.functions.udf
import com.salesforce.op._
import com.salesforce.op.features._
import com.salesforce.op.features.types._
import com.salesforce.op.stages.impl.classification._
import com.salesforce.op.evaluators.Evaluators


In [9]:
import com.salesforce.op.OpWorkflow
import com.salesforce.op.evaluators.Evaluators
import com.salesforce.op.readers.DataReaders

import com.salesforce.op.OpWorkflow
import com.salesforce.op.evaluators.Evaluators
import com.salesforce.op.readers.DataReaders


In [10]:
val conf = new SparkConf().setMaster("local[*]").setAppName("TitanicPrediction")
implicit val spark = SparkSession.builder.config(conf).getOrCreate()

org.apache.spark.sql.SparkSession@7cd77893

In [11]:
case class SimpleRegression (
  population: Double,
  profit: Double)

defined class SimpleRegression


In [12]:
val population = FeatureBuilder.RealNN[SimpleRegression].extract(_.population.toRealNN).asPredictor
val profit = FeatureBuilder.RealNN[SimpleRegression].extract(_.profit.toRealNN).asResponse

Feature(name = profit, uid = RealNN_000000000004, isResponse = true, originStage = FeatureGeneratorStage_000000000004, parents = [], distributions = [])

In [16]:
import spark.implicits._

val trainFilePath = "../src/main/resources/SimpleRegressionDataset/simple_regression.csv"

val trainDataReader = DataReaders.Simple.csvCase[SimpleRegression](
    path = Option(trainFilePath)
)

org.apache.spark.sql.SparkSession$implicits$@3fc56cc7

In [17]:
// check that path exists
scala.reflect.io.File(trainFilePath).exists

true

In [19]:
import com.salesforce.op.stages.impl.tuning.{DataCutter, DataSplitter}
val features = Seq(population).transmogrify()
val randomSeed = 42L
val splitter = DataSplitter(seed = randomSeed)

DataSplitter_000000000007

In [22]:
import com.salesforce.op.stages.impl.regression.RegressionModelsToTry.{OpGBTRegressor, OpRandomForestRegressor}
import com.salesforce.op.stages.impl.regression.RegressionModelSelector

val cutter = DataCutter(reserveTestFraction = 0.2, seed = randomSeed)

val prediction = RegressionModelSelector
      .withCrossValidation(
        dataSplitter = Some(splitter), seed = randomSeed,
        modelTypesToUse = Seq(OpGBTRegressor, OpRandomForestRegressor)
).setInput(profit, features).getOutput()

Feature(name = population-profit_3-stagesApplied_Prediction_000000000011, uid = Prediction_000000000011, isResponse = true, originStage = ModelSelector_000000000011, parents = [RealNN_000000000004,OPVector_000000000006], distributions = [])

In [23]:
val evaluator = Evaluators.Regression().setLabelCol(profit).
      setPredictionCol(prediction)

OpRegressionEvaluator_000000000012

In [24]:
val workflow = new OpWorkflow().setResultFeatures(prediction, profit).setReader(trainDataReader)
val workflowModel = workflow.train()

com.salesforce.op.OpWorkflowModel@710ce57d

In [25]:
val dfScoreAndEvaluate = workflowModel.scoreAndEvaluate(evaluator)
val dfScore = dfScoreAndEvaluate._1.withColumnRenamed("population-profit_3-stagesApplied_Prediction_000000000011",
      "predicted_profit")
val dfEvaluate = dfScoreAndEvaluate._2
println("Evaluate:\n" + dfEvaluate.toString())

dfScore.show(false)

Evaluate:
{
  "RootMeanSquaredError" : 3.22865415217584,
  "MeanSquaredError" : 10.424207634362292,
  "R2" : 0.6531042203541152,
  "MeanAbsoluteError" : 2.3330556193792265
}
+--------------------+-------+---------------------------------------------------------+
|key                 |profit |population-profit_3-stagesApplied_Prediction_000000000011|
+--------------------+-------+---------------------------------------------------------+
|6777483601126752519 |17.592 |[prediction -> 3.706587562620248]                        |
|-3233451928232662667|9.1302 |[prediction -> 2.2725921526937687]                       |
|-2874598320917061466|13.662 |[prediction -> 7.857711177463028]                        |
|-6655228925740401190|11.854 |[prediction -> 4.7661764649096545]                       |
|6173061078876678299 |6.8233 |[prediction -> 2.6774891813962927]                       |
|-640505054168190865 |11.886 |[prediction -> 7.024292748677156]                        |
|6146948818121958743 |4.3

null